<a href="https://colab.research.google.com/github/Ikmalrl/Computer-Security/blob/main/LunarBlaze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import hashlib
from typing import Tuple

# ---------- Implementation Section ----------
class LFSR:
    def __init__(self, size:int, taps:Tuple[int,...], seed:int):
        self.size = size
        self.taps = taps
        self.state = seed & ((1 << size) - 1)

    def clock(self) -> int:
        fb = 0
        for t in self.taps:
            fb ^= (self.state >> (self.size - t)) & 1
        out = self.state & 1
        self.state = ((self.state >> 1) | (fb << (self.size - 1))) & ((1 << self.size) - 1)
        return out

def nonlinear_mix(a:int, b:int, s:int) -> int:
    return (a & (~b) | (~a & ((s >> 3) & 1))) & 1

class KiteStream25:
    NAME = "MangoTango"
    def __init__(self, key:bytes, iv:bytes):
        seed = hashlib.sha256(key + iv).digest()
        s1_seed = int.from_bytes(seed[:4], 'big') ^ 0xA5A5A5A5
        s2_seed = int.from_bytes(seed[4:8], 'big') ^ 0x5A5A5A5A
        self.l1 = LFSR(31, (31, 28), s1_seed)
        self.l2 = LFSR(29, (29, 27), s2_seed)
        self.state = int.from_bytes(seed[8:16], 'big')

    def keystream_byte(self) -> int:
        b = 0
        for i in range(8):
            a = self.l1.clock()
            c = self.l2.clock()
            m = nonlinear_mix(a, c, self.state)
            bit = 1 if (a + c + m) >= 2 else 0
            b = (b << 1) | bit
            if i % 3 == 0:
                self.state ^= ((a << 3) | (c << 7)) & 0xFFFFFFFFFFFFFFFF
        return b

    def keystream(self, n:int) -> bytes:
        return bytes(self.keystream_byte() for _ in range(n))

    def encrypt(self, plaintext:bytes) -> bytes:
        ks = self.keystream(len(plaintext))
        return bytes(p ^ k for p, k in zip(plaintext, ks))

    def decrypt(self, ciphertext:bytes) -> bytes:
        return self.encrypt(ciphertext)  # symmetric


# ---------- Demo Section ----------
if __name__ == "__main__":
    print("\n===== LunarBlaze Stream Cipher =====")

    # User inputs plaintext
    plaintext = input("Enter plaintext: ").encode()

    # Key and IV can be constants or user-provided for testing
    key = b"demo-stream-key-01"
    iv  = b"unique-iv-01"

    # Initialize cipher
    cipher = KiteStream25(key, iv)

    # Encrypt and decrypt
    ciphertext = cipher.encrypt(plaintext)
    cipher2 = KiteStream25(key, iv)
    decrypted = cipher2.decrypt(ciphertext)

    # Display results
    print("\nStatus          : Encryption complete!")
    print("Ciphertext (hex):", ciphertext.hex())
    print("Decrypted text  :", decrypted.decode())

    # Verification
    if decrypted == plaintext:
        print("\nSuccess: Decryption matches with the plaintext.")
    else:
        print("\nError: Decryption failed!")


===== LunarBlaze Stream Cipher =====
Enter plaintext: computer security

Status          : Encryption complete!
Ciphertext (hex): 9c3a37508e0b7b3fd17c0a888799bddbb2
Decrypted text  : computer security

Success: Decryption matches with the plaintext.
